#### Set up
Run this section before anything else

In [1]:
from datetime import datetime, timedelta
import Dataloader_ricequant as dl

import pandas as pd
import rqdatac as rq
import missingno as msno
from constants import *

In [2]:
START_DATE = '20110101'
END_DATE = '20201231'

In [3]:
dl.rq_initialize()
listed_dates = dl.load_listed_dates()

/Users/liguolun/opt/anaconda3/envs/multifactor/lib/python3.7/site-packages/rqdatac/client.py:238: UserWarning: Your account will be expired after  152 days. Please call us at 0755-22676337 to upgrade or purchase or renew your contract.
  "renew your contract.".format(remaining_days))


In [4]:
price_data = dl.load_price_data()

In [5]:
price_data

,688136.XSHG,603105.XSHG,002435.XSHE,002353.XSHE,000544.XSHE,603663.XSHG,000550.XSHE,001896.XSHE,603677.XSHG,002347.XSHE,...,603108.XSHG,002438.XSHE,600401.XSHG,600367.XSHG,003040.XSHE,300297.XSHE,000549.XSHE,600373.XSHG,300283.XSHE,600415.XSHG
date,,,,,,,,,,,,,,,,,,,,,
2021-07-30,34.40,9.82,6.00,37.90,5.92,17.34,18.50,6.79,11.71,4.47,...,10.21,10.87,NaN,12.03,23.02,4.70,NaN,10.76,7.38,4.41
2021-07-29,34.06,9.50,5.79,38.85,5.96,16.82,18.56,6.47,11.37,4.30,...,10.50,10.95,NaN,12.75,22.55,4.66,NaN,10.70,7.32,4.52
2021-07-28,33.91,8.87,5.26,40.08,5.95,15.55,18.45,6.40,11.08,4.12,...,10.17,10.88,NaN,11.98,21.40,4.53,NaN,10.69,6.63,4.53
2021-07-27,33.14,9.33,4.78,40.69,6.12,16.34,19.26,6.92,11.54,4.43,...,10.42,11.30,NaN,11.84,23.10,4.91,NaN,10.38,6.96,4.54
2021-07-26,33.07,10.19,5.04,41.61,6.30,17.66,18.89,6.90,11.99,4.45,...,10.60,11.40,NaN,12.10,23.72,5.03,NaN,10.45,6.86,4.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1991-05-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1991-05-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1991-04-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
def sort_index_and_col(df):
    return df.sort_index(axis=0).reindex(sorted(df.columns), axis=1)

#### Missing value analysis of price data

In [6]:
# backtest 10 years from 2011 to 2020
price_data.index = pd.to_datetime(price_data.index)
price_data = price_data.sort_index(axis=0).reindex(sorted(price_data.columns), axis=1)
backtest_price_data = price_data[START_DATE: END_DATE]

In [50]:
%reset_selective -f ^price_data$

In [7]:
backtest_price_data.shape

(2432, 4560)

In [8]:
# Drop a col if it is all nan
#These cols represent stocks listed after the backtesting period, no lookahead bias
backtest_price_data.dropna(axis=1, how='all',inplace=True)
backtest_price_data.shape

/Users/liguolun/opt/anaconda3/envs/multifactor/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


(2432, 4196)

In [9]:
backtest_price_data.columns

Index(['000001.XSHE', '000002.XSHE', '000004.XSHE', '000005.XSHE',
       '000006.XSHE', '000007.XSHE', '000008.XSHE', '000009.XSHE',
       '000010.XSHE', '000011.XSHE',
       ...
       '688668.XSHG', '688678.XSHG', '688679.XSHG', '688686.XSHG',
       '688698.XSHG', '688699.XSHG', '688777.XSHG', '688788.XSHG',
       '688981.XSHG', '689009.XSHG'],
      dtype='object', length=4196)

In [10]:
backtest_price_data = backtest_price_data.sort_index(axis=0).reindex(sorted(backtest_price_data.columns), axis=1)
backtest_price_data = backtest_price_data.stack(dropna=False).to_frame().rename_axis(['date', 'stock'], axis=0)
backtest_price_data.columns = ['close']

In [11]:
backtest_price_data.shape

(10204672, 1)

In [12]:
stock_names = list(backtest_price_data.columns)

In [13]:
#Run code below if data has not been stored in your local Data folder
# df_is_st = rq.is_st_stock(stock_names, START_DATE, END_DATE)
# df_is_suspended = rq.is_suspended(stock_names, START_DATE, END_DATE)
# df_is_st.to_hdf('./Data/raw_data/filtering_info.h5', key='is_st')
# df_is_suspended.to_hdf('./Data/raw_data/filtering_info.h5', key='is_suspended')

In [14]:
df_is_st = pd.read_hdf('./Data/raw_data/filtering_info.h5', key='is_st')
df_is_suspended = pd.read_hdf('./Data/raw_data/filtering_info.h5', key='is_suspended')

In [15]:
#sorting the dataframe when it is a two-dimensional is much faster than when it later on gets stacked i.e. becomes one-dimensional with multi-index
df_is_st = df_is_st.sort_index().reindex(sorted(df_is_st.columns), axis=1)
df_is_suspended = df_is_suspended.sort_index().reindex(sorted(df_is_suspended.columns), axis=1)

In [16]:
not_st_nor_suspended = (~(df_is_st | df_is_suspended)).stack() #neither ST nor suspended

In [17]:
not_st_nor_suspended

2011-01-04  000001.XSHE     True
            000002.XSHE     True
            000004.XSHE    False
            000005.XSHE     True
            000006.XSHE     True
                           ...  
2020-12-31  688699.XSHG     True
            688777.XSHG     True
            688788.XSHG     True
            688981.XSHG     True
            689009.XSHG     True
Length: 10204672, dtype: bool

In [18]:
backtest_price_data

close
date       stock              
2011-01-04 000001.XSHE   16.02
           000002.XSHE    8.80
           000004.XSHE   12.09
           000005.XSHE    3.75
           000006.XSHE    7.27
...                        ...
2020-12-31 688699.XSHG   55.17
           688777.XSHG  100.20
           688788.XSHG  133.30
           688981.XSHG   57.75
           689009.XSHG   85.82

[10204672 rows x 1 columns]

In [24]:
df_is_listed = pd.DataFrame(True, index=backtest_price_data.index, columns=['is_listed'])

In [26]:
listed_dates = dl.load_listed_dates()
listed_dates = pd.DataFrame(listed_dates.sort_index()).reset_index()
listed_dates.columns = ['stock', 'listed_date']

In [29]:
listed_dates

,stock,listed_date
0,000001.XSHE,1991-04-03
1,000002.XSHE,1991-01-29
2,000003.XSHE,1991-07-03
3,000004.XSHE,1991-01-14
4,000005.XSHE,1991-01-04
...,...,...
4555,688799.XSHG,2021-07-13
4556,688800.XSHG,2021-07-22
4557,688819.XSHG,2021-01-18
4558,688981.XSHG,2020-07-16


In [35]:
df_is_listed = pd.merge(df_is_listed.reset_index(), listed_dates,
     how = 'left', left_on = 'stock', right_on='stock')

In [37]:
df_is_listed['is_listed'] = df_is_listed['date'] >= df_is_listed['listed_date']
df_is_listed = df_is_listed.set_index(['date', 'stock']).sort_index()

In [41]:
df_is_listed

is_listed listed_date
date       stock                             
2011-01-04 000001.XSHE       True  1991-04-03
           000002.XSHE       True  1991-01-29
           000004.XSHE       True  1991-01-14
           000005.XSHE       True  1991-01-04
           000006.XSHE       True  1992-04-27
...                           ...         ...
2020-12-31 688699.XSHG       True  2020-12-18
           688777.XSHG       True  2020-11-24
           688788.XSHG       True  2020-10-22
           688981.XSHG       True  2020-07-16
           689009.XSHG       True  2020-10-29

[10204672 rows x 2 columns]

In [58]:
import pathos
def applyParallel(dfGrouped, func):
    #parrallel computing version of pd.groupby.apply
    with pathos.multiprocessing.ProcessPool(pathos.helpers.cpu_count()) as pool:
        ret_list = pool.map(func, [group for name, group in dfGrouped])
    return pd.concat(ret_list)

# def set_listed_ind(df):
#     #set the boolean indicator for whether this stock is listed or not
#     stock_name = df.index.get_level_values(1)[0]
#     df[df.index.get_level_values(0) >= listed_dates[stock_name]] = True
#     return df
# # df_is_listed = applyParallel(df_is_listed.groupby(level=1), set_listed_ind)

In [43]:
backtest_price_data = backtest_price_data[not_st_nor_suspended.values & df_is_listed['is_listed'].values]

In [44]:
backtest_price_data #more than 1/3 or the data were filtered out

close
date       stock              
2011-01-04 000001.XSHE   16.02
           000002.XSHE    8.80
           000005.XSHE    3.75
           000006.XSHE    7.27
           000009.XSHE   16.94
...                        ...
2020-12-31 688699.XSHG   55.17
           688777.XSHG  100.20
           688788.XSHG  133.30
           688981.XSHG   57.75
           689009.XSHG   85.82

[6664030 rows x 1 columns]

In [63]:
def remove_outlier(df, n=3):
    med = df.median()
    MAD = (df - med ).abs()
    upper_limit = med + n * MAD
    lower_limit = med - n * MAD
    df[df > upper_limit] = upper_limit
    df[df < lower_limit] = lower_limit
    return df

In [65]:
backtest_price_data = applyParallel(backtest_price_data.groupby(level=0), remove_outlier)

In [66]:
def standardize(df):
    return (df - df.mean()) / df.std()
    
backtest_price_data = applyParallel(backtest_price_data.groupby(level=0), standardize)

In [70]:
backtest_price_data = backtest_price_data.fillna(0)

In [92]:
backtest_price_data

close
date       stock                
2011-01-04 000001.XSHE -0.316126
           000002.XSHE -0.690943
           000005.XSHE -0.953108
           000006.XSHE -0.770372
           000009.XSHE -0.268366
...                          ...
2020-12-31 688699.XSHG  0.561833
           688777.XSHG  1.382391
           688788.XSHG  1.985555
           688981.XSHG  0.608847
           689009.XSHG  1.120352

[6664030 rows x 1 columns]

In [72]:
results = dl.load_basic_info()


In [95]:
results[0].columns

Index(['code', 'open', 'high', 'low', 'close', 'change', 'volume', 'money',
       'traded_market_value', 'market_value', 'turnover', 'adjust_price',
       'report_type', 'report_date', 'PE_TTM', 'PS_TTM', 'PC_TTM', 'PB',
       'adjust_price_f'],
      dtype='object')

In [88]:
backtest_market_value_data = pd.concat([result['market_value'] for result in results], axis=1)
backtest_market_value_data.columns = dl.stock_names
backtest_market_value_data = sort_index_and_col(backtest_market_value_data)


In [93]:
backtest_market_value_data

,000001.XSHE,000002.XSHE,000003.XSHE,000004.XSHE,000005.XSHE,000006.XSHE,000007.XSHE,000008.XSHE,000009.XSHE,000010.XSHE,...,688768.XSHG,688777.XSHG,688788.XSHG,688789.XSHG,688793.XSHG,688799.XSHG,688800.XSHG,688819.XSHG,688981.XSHG,689009.XSHG
date,,,,,,,,,,,,,,,,,,,,,
1990-12-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-12-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-12-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-12-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-12-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-26,3.648310e+11,2.481550e+11,NaN,3.682000e+09,2.286000e+09,6.156000e+09,1.504000e+09,5.923000e+09,5.976000e+10,3.296000e+09,...,3.513000e+09,4.512500e+10,1.182900e+10,1.823800e+10,7.409000e+09,4.226000e+09,8.942000e+09,4.960600e+10,4.519440e+11,4.730100e+10
2021-07-27,3.446490e+11,2.342130e+11,NaN,3.745000e+09,2.223000e+09,6.075000e+09,1.462000e+09,5.840000e+09,5.522100e+10,3.214000e+09,...,3.950000e+09,4.644400e+10,1.160500e+10,1.846800e+10,7.149000e+09,4.171000e+09,7.830000e+09,4.647600e+10,5.088320e+11,4.728000e+10
2021-07-28,3.444550e+11,2.380470e+11,NaN,3.245000e+09,2.170000e+09,5.967000e+09,1.465000e+09,5.645000e+09,4.970100e+10,3.206000e+09,...,4.280000e+09,4.702700e+10,1.108700e+10,1.809000e+10,7.397000e+09,4.109000e+09,8.284000e+09,4.403600e+10,4.682200e+11,4.750500e+10


In [91]:
backtest_market_value_data ** 0.5

,000001.XSHE,000002.XSHE,000003.XSHE,000004.XSHE,000005.XSHE,000006.XSHE,000007.XSHE,000008.XSHE,000009.XSHE,000010.XSHE,...,688768.XSHG,688777.XSHG,688788.XSHG,688789.XSHG,688793.XSHG,688799.XSHG,688800.XSHG,688819.XSHG,688981.XSHG,689009.XSHG
date,,,,,,,,,,,,,,,,,,,,,
1990-12-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-12-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-12-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-12-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-12-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-26,604012.417091,498151.583356,NaN,60679.485825,47812.132352,78460.180984,38781.438859,76961.029099,244458.585450,57410.800378,...,59270.566051,212426.457862,108761.206319,135048.139565,86075.548212,65007.691853,94562.148876,222724.044504,672267.803781,217487.930700
2021-07-27,587068.139146,483955.576474,NaN,61196.405123,47148.700937,77942.286341,38236.108589,76419.892698,234991.489208,56692.151132,...,62849.025450,215508.700520,107726.505559,135897.019835,84551.759296,64583.279570,88487.287223,215582.930679,713324.610539,217439.646799
2021-07-28,586902.888049,487900.604632,NaN,56964.901475,46583.258795,77246.359138,38275.318418,75133.215025,222937.210891,56621.550668,...,65421.708935,216857.095803,105294.824184,134499.070629,86005.813757,64101.482042,91016.482024,209847.563722,684266.030137,217956.417662


In [ ]:
# Calculate percentage of data missing for each stock, 
# record all stocks that have more than 50% data missing
missing = backtest_price_data.isnull().sum().reset_index().rename(columns = {0:'missNum'}).sort_values('missNum')
missing['missRate'] = missing['missNum']/backtest_price_data.shape[0]
missing = missing[missing.missRate > 0.5].sort_values(by = 'missRate', ascending=False)
missing

In [ ]:
# Drop entries of stock with more than 50% data missing
removed_code = missing['index'].to_list()
backtest_codes = list(set(backtest_price_data.columns) - set(removed_code))

# FIXME: dropping stock data at the end of 10 yrs testing period is a form of look ahead bias
backtest_price_data = backtest_price_data[backtest_codes] 

In [ ]:
# Store processed price data to HDF files
price_data.to_hdf('full price data.h5',key='price')
backtest_price_data.to_hdf('10-year non-ST price data.h5', key='price')

#### Download factor data

In [ ]:
# Fill in factors intended to download, startdate, and enddate
# to bulk download. Downloaded file is saved to './data/factor'.
# Refer to Ricequant doc for all available factors:
# https://www.ricequant.com/welcome/rqdata 


factor_list = ['OBOS']
startdate = '20201001'
enddate = '20211001'

for factor in []:
    dl.download_factor_data(price_data.columns, factor, '20201001', '20211001') 

#### Missing value analysis of factor data

In [ ]:
dl.load_factor_data('OBOS')

OBOS
order_book_id date            
000001.XSHE   2021-08-31  -4.0
              2021-09-01  -4.0
              2021-09-02  -2.0
              2021-09-03  -2.0
              2021-09-06   0.0
              2021-09-07   0.0
              2021-09-08   0.0
              2021-09-09   0.0
              2021-09-10   2.0
              2021-09-13   2.0
              2021-09-14   0.0
              2021-09-15   0.0
              2021-09-16  -2.0
              2021-09-17  -2.0
              2021-09-22  -4.0
              2021-09-23  -6.0
              2021-09-24  -6.0
              2021-09-27  -4.0
              2021-09-28  -4.0
              2021-09-29  -2.0
              2021-09-30  -2.0